In [396]:
# !pip install numpy
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample

In [397]:
import warnings
warnings.filterwarnings("ignore")


In [398]:
# для оптимизации значений гаммы

from scipy.optimize import minimize_scalar

In [399]:
class GradientBoosting:
    def __init__(
        self, 
        base_model_class: object = DecisionTreeRegressor,
        base_model_params: dict = {'max_depth': None}, 
        n_estimators: int = 10,
        learning_rate: float = 0.1
    ):
        
        
        self.base_model_class = base_model_class
        self.base_model_params = base_model_params
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        
        # list for optimal gammas at each iteration
        self.gammas = []
        
        # list for base models
        self.models = []
        
        
        # Я создала 2 новые функции: одна для подсчета градиента (просто ради удобства),
        # Вторая для подсчета функции потерь с учетом гаммы (ее я потом оптимизировала в функции find_optimal_gamma)
        
    def gradient_calc(self, y_true, y_pred, n_estimators):
    
        grad = (-2/self.n_estimators)*(y_true-y_pred)
    
        return grad
    
    def gamma_loss(self, y, y_pred_old, y_pred_new, gamma):
        
        return sum((1/self.n_estimators)*np.transpose((y_pred_old + gamma*(y_pred_new-y_pred_old) - y))*(y_pred_old + gamma*(y_pred_new-y_pred_old) - y))
    
    def find_optimal_gamma(self, y, y_pred_old, y_pred_new):
        
        min_loss = minimize_scalar(lambda gamma: self.gamma_loss(y, y_pred_old, y_pred_new, gamma),bounds=(0,1))
        
        return min_loss.x
    
    
    def _fit_base_model(self, X: np.ndarray, y: np.array):
        
        model = self.base_model_class(**self.base_model_params)
        model.fit(X,y)
        
        return model
    
        
    def fit(self, X: np.ndarray, y: np.array):
        
        # Отдельно обучила первую модель из композиции, задала первый прогноз ее прогнозом
        
        initial_pred_model = self.base_model_class(**self.base_model_params)
        initial_pred = initial_pred_model.fit(X,y)
        
        y_pred = initial_pred_model.predict(X)
        
        self.models.append(initial_pred)
        
        # Итеративно обучила каждую модель композиции на подвыборках градиентов соответствующих данных

        for iteration in range(self.n_estimators): 
            

            grad = self.gradient_calc(y, y_pred, self.n_estimators)
            
            X_k, grad_k = resample(X, grad, n_samples = len(X)//(self.n_estimators))
        
            model = self._fit_base_model(X_k, grad_k)
        
            pred_grad = model.predict(X)
            
            # Для оптимизации гаммы я отдельно сохраняла старые и новые прогнозы (без гаммы)
            
            y_pred_old = y_pred
            y_pred_new = y_pred + self.learning_rate*pred_grad
            
            # Потом использовала их для оптимизации как аргументы функции
            
            
            gamma = self.find_optimal_gamma(y, y_pred_old, y_pred_new)
            
            self.gammas.append(gamma)
            
            # Полноценно обновляла прогноз через новые предсказания (уже с гаммой)
            
            y_pred += gamma*self.learning_rate * pred_grad
            
            self.models.append(model)
        
        return self.models
        
       
        
    def predict(self, X: np.ndarray):
        
        # Изначальное предсказание - просто предсказание первого дерева
        
        final_pred = self.models[0].predict(X)
        
        # Потом итеративно проходимся по всем деревьям, суммируем предсказания
    
        for iteration in range(1,len(self.models)):
        
            model = self.models[iteration]
            
            temp_pred = model.predict(X)
            
            final_pred += self.learning_rate*temp_pred
    
        return final_pred

Проверьте вашу реализацию на бостонском датасете. Подберите оптимальные гиперпараметры, чтобы победить RandomForestRegressor (не меняйте параметры сида).

In [400]:
housing = fetch_california_housing()
X = housing.data[:5000]
y = housing.target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [401]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_features=4, n_estimators=640, random_state=19052019)

rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test))

0.1673784076585523

In [382]:
GB = GradientBoosting()
GB.fit(X_train, y_train)
mean_squared_error(y_test, GB.predict(X_test))

0.33684880318576

In [395]:
# Попробуем подобрать гиперпараметры (это окошко можно не запускать, я делала его для своего удобства)


final_error = 0.4
for lr in range(100, 10, -5):
    for n in range (2, 30, 1):
        for depth in range(6,10):
            GB = GradientBoosting(learning_rate = lr/10000, n_estimators = n, base_model_params = {'max_depth': depth})
            GB.fit(X_train, y_train)
            current_error = mean_squared_error(y_test, GB.predict(X_test))
            if final_error > current_error:
                final_error = current_error
                optimal_lr = lr/10000
                optimal_n = n
                optimal_depth = depth
print(final_error, optimal_lr, optimal_n, optimal_depth)

0.24077874106988897 0.003 27 7


In [ ]:
# Победить случайный лес у меня так и не получилось, вышло только приблизить
# значение ошибки к 0.167 - минимальная ошибка на кастомном бустинге при подборе параметров это примерно 0.24
# этого вышло добиться с гиперпараметрами: lr = 0.003, n_estimators = 27, max_depth = 7

In [410]:
GB = GradientBoosting(learning_rate = 0.003, n_estimators = 27, base_model_params = {'max_depth': 7})
GB.fit(X_train, y_train)
print(mean_squared_error(y_test, GB.predict(X_test)))

0.24927436890619278


In [409]:
# Еще ради интереса провела эксперимент с xgboost 

from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X, y)
mean_squared_error(y_test, model.predict(X_test))

0.0161254278818677

Сравниваем подходы


In [411]:
!wget  -O 'bank_data.csv' -q 'https://www.dropbox.com/s/uy27mctxo0gbuof/bank_data.csv?dl=0'

python(34316) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [412]:
df = pd.read_csv('bank_data.csv')
df.sample(5)

f_names = ['age', 'job', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

In [ ]:
# Решите задачу предсказания возвращения кредита методами, перечисленными ниже:

- Случайный лес
- Бэггинг на деревьях (поставьте для базовых деревьев min_samples_leaf=1)
- Бэггинг, у которого базовой моделью является бустинг с большим числом деревьев (> 100)
- Бэггинг на логистических регрессиях

Используйте логистическую регрессию, случайный лес, `GradientBoostingClassifier` и `BaggingClassifier` из `sklearn`.

1) Какая из моделей имеет лучшее качество? С чем это связано?

2) Какая из моделей сильнее всего переобучается?


In [413]:

X = df[f_names].values

y_nr = 1 * (df['y'].values)

y = y_nr.reshape(-1,1)

In [414]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()

X_enc = enc.fit_transform(X)
y_enc = enc.fit_transform(y)


In [415]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=0.2, random_state=42)


In [416]:
from sklearn.metrics import log_loss

In [417]:
# Случайный лес
    
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train, np.ravel(y_train))
log_loss(np.ravel(y_test), rf.predict(X_test))

3.759129297313985

In [418]:
# Бэггинг на деревьях (min_samples_leaf = 1)

from sklearn.ensemble import BaggingClassifier

bc_dt = BaggingClassifier(base_estimator=DecisionTreeClassifier(min_samples_leaf = 1))

bc_dt.fit(X_train, np.ravel(y_train))
log_loss(np.ravel(y_test), bc_dt.predict(X_test))

4.4104493282649555

In [419]:
# Бэггинг, у которого базовой моделью является бустинг с большим числом деревьев (> 100)

from sklearn.ensemble import GradientBoostingClassifier

bc_gb = BaggingClassifier(base_estimator=GradientBoostingClassifier(n_estimators = 101))

bc_gb.fit(X_train, np.ravel(y_train))
log_loss(np.ravel(y_test), bc_gb.predict(X_test))

3.7963469447250633

In [420]:
# Бэггинг на логистических регрессиях (увеличила количество итераций из-за рекомендации предупреждения*)

from sklearn.linear_model import LogisticRegression

bc_lr = BaggingClassifier(base_estimator=LogisticRegression(max_iter=1000))

bc_lr.fit(X_train, np.ravel(y_train))
log_loss(np.ravel(y_test), bc_lr.predict(X_test))

4.5779352038785195

In [421]:
# Проверка переобучения:

# 1. Случайный лес


print("Ошибка на тренировочной выборке:")
print(log_loss(np.ravel(y_train), rf.predict(X_train)))
print("Ошибка на тестовой выборке:")
print(log_loss(np.ravel(y_test), rf.predict(X_test)))

# Переобучение есть, и оно сильное

Ошибка на тренировочной выборке:
9.992007221626413e-16
Ошибка на тестовой выборке:
3.759129297313985


In [422]:
# 2. Бэггинг на деревьях

print("Ошибка на тренировочной выборке:")
print(log_loss(np.ravel(y_train), bc_dt.predict(X_train)))
print("Ошибка на тестовой выборке:")
print(log_loss(np.ravel(y_test), bc_dt.predict(X_test)))

# Переобучение есть, но чуть слабее, чем у случайного леса

Ошибка на тренировочной выборке:
0.2791416181610766
Ошибка на тестовой выборке:
4.4104493282649555


In [423]:
# 3. Бэггинг на бустингах деревьев

print("Ошибка на тренировочной выборке:")
print(log_loss(np.ravel(y_train), bc_gb.predict(X_train)))
print("Ошибка на тестовой выборке:")
print(log_loss(np.ravel(y_test), bc_gb.predict(X_test)))

# Переобучения нет

Ошибка на тренировочной выборке:
3.6056004702971554
Ошибка на тестовой выборке:
3.7963469447250633


In [424]:
# 4. Бэггинг на логистических регрессиях

print("Ошибка на тренировочной выборке:")
print(log_loss(np.ravel(y_train), bc_lr.predict(X_train)))
print("Ошибка на тестовой выборке:")
print(log_loss(np.ravel(y_test), bc_lr.predict(X_test)))

# Переобучения нет

Ошибка на тренировочной выборке:
4.69889471205243
Ошибка на тестовой выборке:
4.5779352038785195


In [ ]:
"""

Лучше всего показывает себя Случайный лес.

Это происходит потому, что он учится на подвыборках, то есть вынужденно добавляет разнообразия и "случайности".
Деревья получаются независимыми друг от друга, в отличие от моделей бэггинга.
В данной задаче классификации это позволяет учитывать все признаки, не привязывать их друг к другу, учитывать все
критерии, которые могут влиять на результирующий таргет.

Сильнее всего переобучается Случайный лес.

Возможно, по умолчанию модель слишком сложна для этой задачи, и деревья чересчур глубокие,
подстраиваются под тренировочную выборку слишком сильно. К тому же в дефолтной модели у нас не оптимизрованы
и другие гиперпараметры, это так же может влиять

"""

Сравнение встроенного бустинга xgboos

In [425]:
#conda install -c conda-forge xgboost

python(34379) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


failed

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [427]:
from xgboost import XGBClassifier

model = XGBClassifier(eval_metric='logloss')

model.fit(X_train, y_train)
log_loss(y_test, model.predict(X_test))


3.7777325203909715

In [ ]:
from sklearn.model_selection import GridSearchCV


parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

clf = GridSearchCV(model, parameters, scoring='neg_log_loss')

clf.fit(X_train, y_train)

print((-1)*clf.best_score_, clf.best_params_)

In [ ]:
# NB!: У нас выходит отрицательная логистическая функция потерь, но это нормально, тк такова особенность
# работы АПИ скоринга Sklearn, он по умолчанию нацелен на максимизацию функционала качества, и в случае,
# где нужно минимизировать функционал, он домножает его на -1. Истинное значение логистической функции потерь
# это просто модуль получившегося значения. То есть ошибка равна примерно 0.27

In [ ]:
"""
ОТВЕТЫ НА ВОПРОСЫ:

Получилось явно гораздо лучше, чем с моделями выше. LogLoss = 3.6 vs LogLoss = 0.27 !

Потому что XGBoost итеративно оптимизирует гиперпараметры с каждым шагом, лучше справляется с отсутствием
баланса в классах и учится на собственных ошибках предыдущей итерации на каждом шаге.


"""